### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

#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


In [1]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
!pip install langchain_groq

  Using cached langchain_core-0.3.44-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain_core-0.3.44-py3-none-any.whl (415 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40


In [3]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019435832F20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019435868310>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
!pip install langchain_core

In [6]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)
result

AIMessage(content='The translation of "Hello, How are you?" in French is:\n\n**Bonjour, comment allez-vous ?** \n\nHere\'s a breakdown:\n\n* **Bonjour:** Hello\n* **Comment:** How\n* **allez-vous:** are you (formal, polite)\n\n\nLet me know if you\'d like to know other ways to say "How are you?" in French! 😊\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 21, 'total_tokens': 106, 'completion_time': 0.154545455, 'prompt_time': 0.002125015, 'queue_time': 0.23373463, 'total_time': 0.15667047}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f0f2552e-9d5e-44a7-8e3e-cba807117c9d-0', usage_metadata={'input_tokens': 21, 'output_tokens': 85, 'total_tokens': 106})

In [7]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'The translation of "Hello, How are you?" in French is:\n\n**Bonjour, comment allez-vous ?** \n\nHere\'s a breakdown:\n\n* **Bonjour:** Hello\n* **Comment:** How\n* **allez-vous:** are you (formal, polite)\n\n\nLet me know if you\'d like to know other ways to say "How are you?" in French! 😊\n'

In [8]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here are a couple of ways to say "Hello, How are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal)\n\n\nLet me know if you\'d like other variations! 😊\n'

In [9]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [11]:
result=prompt.invoke({"language":"French","text":"Hello"})
result

ChatPromptValue(messages=[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [12]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [13]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'